In [1]:
from datasets import concatenate_datasets, load_from_disk
from transformers import BasicTokenizer, Trainer, EarlyStoppingCallback
import kagglehub
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
#my_glove = kagglehub.dataset_download("takuok/glove840b300dtxt")
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [3]:
#GLOVE_FILE = f"{my_glove}/glove.840B.300d.txt"
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "dbpedia"

In [4]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits", f"~/data/{DATASET}/test-logits", f"~/data/{DATASET}/train-logits-augmented"]])
tokenizer = BasicTokenizer(do_lower_case=True)

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [6]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))
all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

In [7]:
vocab = base.get_vocab(all_data_tokens)

In [ ]:
word_index = dict(zip(vocab, range(len(vocab))))

In [9]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


In [ ]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

691158


In [11]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 212978 words (478180) misses


In [12]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

In [13]:
train_padded_data = list(map(lambda x: base.padd(x,300), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,300), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,300), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,300), all_train_data_index))

In [14]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

In [15]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [16]:
print(model)

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)


In [17]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base", logging_dir=f"~/logs/{DATASET}/bilstm-base", lr=.0001,  epochs=3, batch_size=128)

In [18]:
base.reset_seed()

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.235600,0.070291,0.979714,0.979851,0.979714,0.979752
2,0.064100,0.058508,0.983259,0.983298,0.983259,0.983255
3,0.050900,0.051877,0.985464,0.985466,0.985464,0.985465


TrainOutput(global_step=10500, training_loss=0.11686382330031622, metrics={'train_runtime': 408.1556, 'train_samples_per_second': 3292.862, 'train_steps_per_second': 25.725, 'total_flos': 0.0, 'train_loss': 0.11686382330031622, 'epoch': 3.0})

In [21]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

In [22]:
trainer.evaluate(test_data)

{'eval_loss': 0.051313064992427826,
 'eval_accuracy': 0.9853571428571428,
 'eval_precision': 0.9853566836590295,
 'eval_recall': 0.9853571428571427,
 'eval_f1': 0.9853550685922283,
 'eval_runtime': 13.6094,
 'eval_samples_per_second': 5143.513,
 'eval_steps_per_second': 40.193,
 'epoch': 3.0}

In [23]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base.pth")

In [24]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [25]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill", lr=.0001,  epochs=3, batch_size=128, lambda_param=.75, temp=5)

In [26]:
base.reset_seed()

In [27]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.111900,0.196027,0.979625,0.979671,0.979625,0.979609
2,0.220700,0.145268,0.982750,0.982789,0.982750,0.982723
3,0.179200,0.123894,0.984563,0.984548,0.984562,0.984548


TrainOutput(global_step=10500, training_loss=0.5039353957403274, metrics={'train_runtime': 408.8537, 'train_samples_per_second': 3287.24, 'train_steps_per_second': 25.682, 'total_flos': 0.0, 'train_loss': 0.5039353957403274, 'epoch': 3.0})

In [29]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

In [30]:
trainer.evaluate(test_data)

{'eval_loss': 0.12667638063430786,
 'eval_accuracy': 0.9838285714285714,
 'eval_precision': 0.9838122501524115,
 'eval_recall': 0.9838285714285713,
 'eval_f1': 0.9838068196482092,
 'eval_runtime': 13.5726,
 'eval_samples_per_second': 5157.469,
 'eval_steps_per_second': 40.302,
 'epoch': 3.0}

In [31]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill.pth")

In [32]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [33]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-aug", logging_dir=f"~/logs/{DATASET}/bilstm-base-aug", lr=.0001,  epochs=3, batch_size=128)

In [34]:
base.reset_seed()

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.209000,0.062948,0.981973,0.982077,0.981973,0.982000
2,0.077300,0.052072,0.985232,0.985311,0.985232,0.985253
3,0.063000,0.048477,0.986455,0.986452,0.986455,0.986453


TrainOutput(global_step=20616, training_loss=0.11643917985234466, metrics={'train_runtime': 736.4454, 'train_samples_per_second': 3583.056, 'train_steps_per_second': 27.994, 'total_flos': 0.0, 'train_loss': 0.11643917985234466, 'epoch': 3.0})

In [37]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

In [38]:
trainer.evaluate(test_data)

{'eval_loss': 0.0469534657895565,
 'eval_accuracy': 0.9866,
 'eval_precision': 0.9865947138409358,
 'eval_recall': 0.9866,
 'eval_f1': 0.9865959405631654,
 'eval_runtime': 13.6735,
 'eval_samples_per_second': 5119.398,
 'eval_steps_per_second': 40.004,
 'epoch': 3.0}

In [39]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base-aug.pth")

In [40]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [41]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-aug", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill-aug", lr=.0001,  epochs=3, batch_size=128, lambda_param=.75, temp=5)

In [42]:
base.reset_seed()

In [43]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.891500,0.155760,0.981625,0.981631,0.981625,0.981613
2,0.247300,0.121272,0.984795,0.984796,0.984795,0.984791
3,0.203400,0.109529,0.985286,0.985266,0.985286,0.985270


TrainOutput(global_step=20616, training_loss=0.44738218272708397, metrics={'train_runtime': 737.7634, 'train_samples_per_second': 3576.655, 'train_steps_per_second': 27.944, 'total_flos': 0.0, 'train_loss': 0.44738218272708397, 'epoch': 3.0})

In [45]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

In [46]:
trainer.evaluate(test_data)

{'eval_loss': 0.11187935620546341,
 'eval_accuracy': 0.9852142857142857,
 'eval_precision': 0.9851948305805569,
 'eval_recall': 0.9852142857142857,
 'eval_f1': 0.9851975329023206,
 'eval_runtime': 14.3481,
 'eval_samples_per_second': 4878.69,
 'eval_steps_per_second': 38.123,
 'epoch': 3.0}

In [47]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill-aug.pth")